<a href="https://colab.research.google.com/github/Almonfrey/MAI-Course/blob/main/class6_practical_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [ ]:
# Standard imports for data analysis
import pandas as pd  # Data processing
import numpy as np  # Numerical computing
from sklearn.model_selection import train_test_split

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Matplotlib configuration for inline display (Jupyter only)
%matplotlib inline

Load data

In [ ]:
# Load the housing dataset and display the first 5 rows
df = pd.read_csv("data/us_house_sales.csv")
print(df.head())

Data formatting

In [ ]:
import pandas as pd

# Display current data types before formatting
print('Data types BEFORE formatting:')
print(df.dtypes)

print(df.head())

# 1. Price - remove $ and commas, convert to float
df['Price'] = pd.to_numeric(
    df['Price'].astype(str).str.replace(r'[\$,]', '', regex=True),
    errors='coerce'
).astype(float)

print(df.head())

# 2. Bedrooms - extract number, allow NaN, use nullable Int type
df['Bedrooms'] = pd.to_numeric(
    df['Bedrooms'].str.extract(r'(\d+)')[0],
    errors='coerce'
).astype(int)

# 3. Bathrooms - same as bedrooms
df['Bathrooms'] = pd.to_numeric(
    df['Bathrooms'].str.extract(r'(\d+)')[0],
    errors='coerce'
).astype(int)

# 4. Area (Sqft) - remove text and commas, convert to float
df['Area (Sqft)'] = pd.to_numeric(
    df['Area (Sqft)'].str.replace(r'[^\d.]', '', regex=True),
    errors='coerce'
).astype(float)

# 5. Lot Size - same as area
df['Lot Size'] = pd.to_numeric(
    df['Lot Size'].str.replace(r'[^\d.]', '', regex=True),
    errors='coerce'
).astype(float)

# Verify formatting results
print('\nData types AFTER formatting:')
print(df[['Price', 'Bedrooms', 'Bathrooms', 'Area (Sqft)', 'Lot Size']].dtypes)

Data cleaning

Check for duplicate records

In [ ]:
# Check for duplicate records

# Number of duplicate records BEFORE removal
num_duplicates_before = df.duplicated().sum()
print(f'\nNumber of duplicate records BEFORE removal: {num_duplicates_before}')

# Remove duplicates
df.drop_duplicates(inplace=True)

# Number of duplicate records AFTER removal (should be 0)
num_duplicates_after = df.duplicated().sum()
print(f'Number of duplicate records AFTER removal: {num_duplicates_after}')

Split Dataset

In [ ]:
# Splitting data
train_set, temp_set = train_test_split(df, test_size=0.3, random_state=42)
val_set, test_set = train_test_split(temp_set, test_size=0.5, random_state=42)

print(f"Train set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Test set size: {len(test_set)}")

Missing value treatment

In [ ]:
# Missing Value Treatment
print('Missing values BEFORE cleaning:')
print(train_set.isnull().sum())

# Strategy for each column:
median_year = train_set['Year Built'].median()
train_set.fillna({'Lot Size': train_set['Area (Sqft)'], 'Year Built': median_year}, inplace=True)
train_set.dropna(subset=['Price', 'Area (Sqft)', 'Property Type'], inplace=True)

print('Missing values AFTER cleaning:')
print(train_set.isnull().sum())

Removing outliers

In [ ]:
# Outlier Detection using the IQR method

# Calculate the first (Q1) and third quartiles (Q3) of the 'Price' column
Q1 = train_set['Price'].quantile(0.25)
Q3 = train_set['Price'].quantile(0.75)

# Compute the Interquartile Range (IQR)
IQR = Q3 - Q1

# Define lower and upper bounds for detecting outliers
price_lower_bound = Q1 - 1.5 * IQR
price_upper_bound = Q3 + 1.5 * IQR

# Print the calculated bounds for reference
print(f'\nPrice bounds for outlier detection: Lower: {price_lower_bound:,.2f}, Upper: {price_upper_bound:,.2f}')

# Count outliers BEFORE filtering
outliers_before = train_set[(train_set['Price'] < price_lower_bound) | (train_set['Price'] > price_upper_bound)]
num_outliers_before = len(outliers_before)
print(f'Number of outliers detected before filtering: {num_outliers_before}')

# Filter out the outliers
train_set = train_set[(train_set['Price'] >= price_lower_bound) & (train_set['Price'] <= price_upper_bound)]

# Count outliers AFTER filtering (should be zero)
outliers_after = train_set[(train_set['Price'] < price_lower_bound) | (train_set['Price'] > price_upper_bound)]
num_outliers_after = len(outliers_after)
print(f'Number of outliers detected after filtering: {num_outliers_after}')

Data validity check

In [ ]:
# Data Validity Checks per category

# Number of invalid entries BEFORE filtering
print("Invalid entries BEFORE filtering:")
print(f"Price: {(train_set['Price'] <= 0).sum()} invalid entries")
print(f"Area (Sqft): {(train_set['Area (Sqft)'] <= 0).sum()} invalid entries")
print(f"Bedrooms: {(train_set['Bedrooms'] <= 0).sum()} invalid entries")
print(f"Year Built: {(train_set['Year Built'] <= 1800).sum()} invalid entries")

# Apply all validity conditions to filter the dataframe
train_set = train_set[(train_set['Price'] > 0) &
        (train_set['Area (Sqft)'] > 0) &
        (train_set['Bedrooms'] > 0) &
        (train_set['Year Built'] > 1800)]

# Number of invalid entries AFTER filtering (should be zero)
print("\nInvalid entries AFTER filtering:")
print(f"Price: {(train_set['Price'] <= 0).sum()} invalid entries")
print(f"Area (Sqft): {(train_set['Area (Sqft)'] <= 0).sum()} invalid entries")
print(f"Bedrooms: {(train_set['Bedrooms'] <= 0).sum()} invalid entries")
print(f"Year Built: {(train_set['Year Built'] <= 1800).sum()} invalid entries")

Final check

In [ ]:
# Final missing value check
print('\nMissing values AFTER cleaning:')
print(train_set.isnull().sum())